In [108]:
import mysql.connector
import pandas as pd

In [104]:
class users:
    cnxn = mysql.connector.connect(host='localhost', password='root', user='root', database = "testdb")
    def add_users():
        name = input("enter your name :")
        add = input("enter your current address :" )
        phone = input("enter your phone number :")
        crs = cnxn.cursor()
        crs.execute(f"call add_user('{name}', '{add}', '{phone}')")
        cnxn.commit()
        pas = crs.execute("select password from account_user where user_id = (select max(user_id) from account_user)")
        password = pas.fetchone()[0]
        return f" user with the name {name} is created with the password {password}"
    

    def users_info():
        usercode = int(input("enter your usercode here :"))
        crs = cnxn.cursor()
        qry = f'''select a.user_id, name, address, phone, balance from account_user as a join account_transactions as t
        on a.user_id = t.user_id where a.user_id = {usercode} order by Date_time desc limit 1'''
        df = pd.read_sql_query(qry, con = cnxn)
        return f"Name of the user is {df['name'][0]} and belongs to {df['address'][0]} having phone number {df['phone'][0]} and the current balance is {df['balance'][0]}"
    
    def change_add():
        qry = "select * from account_user"
        df = pd.read_sql_query(qry, con = cnxn)
        usercode = int(input("enter your usercode : "))
        password =  df[df['user_id'] == usercode]['password'][0]
        pas = input("enter your password :")
        
        if password == pas:
            add = input("enter your new address :")
            crs = cnxn.cursor()
            crs.execute(f"call update_address('{add}','{usercode}')")
            cnxn.commit()
            return f" user's address has been changed with the new address {add} with the code {usercode}"
        else:
            return "please enter a valid password"
        
    def change_phone():
        qry = "select * from account_user"
        df = pd.read_sql_query(qry, con = cnxn)
        usercode = int(input("enter your usercode : "))
        password =  df[df['user_id'] == usercode]['password'][0]
        pas = input("enter your password :")
        
        if password == pas:
            phone = input("enter your new phone number :")
            crs = cnxn.cursor()
            crs.execute(f"call update_phone('{phone}','{usercode}')")
            cnxn.commit()
            return f" user's Phone number has been changed with the new phone number {phone} with the code {usercode}"
        else:
            return "please enter a valid password"
        
    
    def change_name():
        qry = "select * from account_user"
        df = pd.read_sql_query(qry, con = cnxn)
        usercode = int(input("enter your usercode : "))
        password =  df[df['user_id'] == usercode]['password'][0]
        pas = input("enter your password :")
        
        if password == pas:
            name = input("enter your new name :")
            crs = cnxn.cursor()
            crs.execute(f"call update_name('{name}','{usercode}')")
            cnxn.commit()
            return f" user's name has been changed with the new name {name} with the code {usercode}"
        else:
            return "please enter a valid password"   
    

In [122]:
class balance:
    cnxn = mysql.connector.connect(host='localhost', password='root', user='root', database = "testdb")
    def check_balance():
        usercode = int(input("enter your usercode here : "))
        crs = cnxn.cursor()
        qry = f'''select balance from account_user as a join account_transactions as t on a.user_id = t.user_id
        where a.user_id = {usercode} order by Date_time desc limit 1'''
        df = pd.read_sql_query(qry, con = cnxn)
        return f"your current balance is {df['balance'][0]}"
    
    def deposit():
        crs = cnxn.cursor()
        usercode = int(input("enter your usercode here :"))
        amount = int(input("enter the deposit amount :"))
        crs.execute(f"call deposit({amount}, {usercode});")
        cnxn.commit()
        qry = f'''select name,balance from account_user as a join account_transactions as t on a.user_id = t.user_id
        where a.user_id = {usercode} order by Date_time desc limit 1'''
        df = pd.read_sql_query(qry, con = cnxn)
        return f"user with the name {df['name'][0]} credit amount of rupee {amount} and the current balance is {df['balance'][0]}"
       
    def withdraw():
        crs = cnxn.cursor()
        usercode = int(input("enter your usercode here :"))
        pas = input("enter your password here :")
        qry = f'''select a.user_id,name,balance, password from account_user as a join account_transactions as t on a.user_id = t.user_id
                where a.user_id = {usercode} order by Date_time desc limit 1'''
        df = pd.read_sql_query(qry, con = cnxn)
        password =  df[df['user_id'] == usercode]['password'][0]
        if pas == password:
            amount = int(input("enter the amount :"))
            if amount <= df[df['user_id'] == usercode]['balance'][0]:
                crs.execute(f"call withdraw({amount}, {usercode});")
                cnxn.commit()
                return f"user with the name {df['name'][0]} has debited amount of rupee {amount} and the current balance is {df['balance'][0]-amount}"
            else:
                return "not having sufficient balance into your account"
        else:
            return "please enter a valid password"